In [1]:
import pandas as pd
import pandas_datareader as pdr
import datetime
import yfinance as yf
import numpy as np
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import matplotlib.pyplot as plt
import xlrd

pd.options.mode.chained_assignment = None
np.set_printoptions(suppress=True)

#FEATURES
#Prediction? Number of days trending upwards? Percentage increase?
#Sell after a 5% increase?
#Dissect what all of this means.
#I want to get a precision / recall curve
#Set rounding threshold lower
#Confidence variables?
#Try other models?
#Try other stocks besides stocks that tend to just go up?
#Predict rise in values rather than just simple classification?
#Set rise rate higher / faster?
#Invest money based on confidence / probability?
#We call more about precision & accuracy than recall
#Remove unnecessary features, prevent overfitting?
#Paper trade over previous data...see how we would have fared?
#Automate all of this?
#Format notebook
#Create github
#Check my math & calculations?
#Train, test, validate data (training data, testing data, stock not involved in the training) Validate on CSCO
#NaN values a problem?
#Overfitting based on general bullish tech stocks?
#Rise 5% within 10 days or AT 10 days?
#get model size / kernel size

#BUGS
#RSI 5 sometimes is NaN?
#Check calculations: 9 or 10 days? Including or not including?
#Normalze MACD?

In [2]:
stock_symbols = ["KR", "JNJ", "WMT", "CPB", "KO", "IP"] 
stock_data = pd.DataFrame(columns=["Ticker","Date","Open","High","Low","Close"])

In [3]:
for stock in stock_symbols:
    y_finance_data = yf.Ticker(stock).history(start=(datetime.date.today()-datetime.timedelta(days=3650)).strftime("%Y-%m-%d")).reset_index()
    y_finance_data.insert(0,"Ticker", stock)
    y_finance_data = y_finance_data[['Ticker', 'Date','Open','High','Low','Close']]
    stock_data = pd.concat([stock_data, y_finance_data])
stock_data = stock_data.rename(columns={"Ticker":"TICKER", "Date":"DATE", "Open":"OPEN", "High":"HIGH", "Low":"LOW", "Close":"CLOSE"})

In [4]:
def RSI_calc(start, lookback, df, date_time=True):
        if date_time:
            start = start.strftime("%Y-%m-%d")
        if ((df.index[df["DATE"]==start][0])-lookback) < 0:
            return None
        df = df[(df.index[df["DATE"]==start][0])-lookback:df.index[df["DATE"]==start][0]]
        
        df['GAIN_LOSS'] = df.apply(lambda row : (row["CLOSE"]-row["OPEN"]) / row["OPEN"] * 100, axis=1)
        avg_gain = df.loc[df['GAIN_LOSS'] >= 0]["GAIN_LOSS"].mean()
        avg_loss = df.loc[df['GAIN_LOSS'] < 0]["GAIN_LOSS"].mean() * -1
        RSI = 100 - (100 / (1 + (avg_gain/avg_loss)))
        return RSI

In [5]:
def daily_return(start, lookback, df, date_time=True):
        if date_time:
            start = start.strftime("%Y-%m-%d")
        if ((df.index[df["DATE"]==start][0])-lookback) < 0:
            return None
        df = df[(df.index[df["DATE"]==start][0])-lookback:df.index[df["DATE"]==start][0]+1] #Include day?
        df['daily_change'] = df.apply(lambda row: ((row['CLOSE'] - row['OPEN']) / row['OPEN']) * 100, axis=1)
        avg_daily_return = df['daily_change'].mean()
        return avg_daily_return

In [6]:
def weekly_return(start, lookback, df, date_time=True): #Check math here!!
        if date_time:
            start = start.strftime("%Y-%m-%d")
        if ((df.index[df["DATE"]==start][0])-lookback) < 0:
            return None
        df = df[(df.index[df["DATE"]==start][0])-lookback:df.index[df["DATE"]==start][0]+1] #Include day?
        df = df.iloc[::5, :]
        df = df.reset_index()
        weekly_change_sum = 0
        for i in range(1, len(df)): #Would really like to not be hacky about this...but oh well
            weekly_change_sum += ((df.iloc[i]['CLOSE'] - df.iloc[i-1]['CLOSE']) / df.iloc[i-1]['OPEN']) * 100
        weekly_change_avg = weekly_change_sum / len(df)-1
        return weekly_change_avg

In [7]:
def monthly_return(start, lookback, df, date_time=True):
        if date_time:
            start = start.strftime("%Y-%m-%d")
        if ((df.index[df["DATE"]==start][0])-lookback) < 0:
            return None
        if(lookback < 30):
            return None
        df = df[(df.index[df["DATE"]==start][0])-lookback:df.index[df["DATE"]==start][0]+1] #Include day?
        df = df.iloc[::30, :]
        df = df.reset_index()
        monthly_change_sum = 0
        for i in range(1, len(df)): #Would really like to not be hacky about this...but oh well
            monthly_change_sum += ((df.iloc[i]['CLOSE'] - df.iloc[i-1]['CLOSE']) / df.iloc[i-1]['OPEN']) * 100
        monthly_change_avg = monthly_change_sum / len(df)-1
        return monthly_change_avg

In [8]:
def EMA(start, lookback, df):
        df = df[(df.index[df["DATE"]==start][0])-lookback:df.index[df["DATE"]==start][0]+1]
        df = df.reset_index()
        SMA = (df['CLOSE'][0:lookback-1].sum()) / lookback
        k = 2 / (lookback + 1)
        EMA = (df.loc[df['DATE']==start]['CLOSE'] * k) + (SMA * (1-k))
        return EMA
    
def MACD(start, lookback1, lookback2, df, date_time=True):
        if date_time:
            start = start.strftime("%Y-%m-%d")
        if ((df.index[df["DATE"]==start][0])-lookback2) < 0:
            return None
        EMA_short = EMA(start, lookback1, df)
        EMA_long = EMA(start, lookback2, df)
        return float(EMA_short) - float(EMA_long)

In [9]:
#df = stock_data.loc[stock_data["TICKER"]=="AAPL"]
#MACD("2020-06-30", 10, 30, df, False)

In [10]:
def ratio_avg_to_close(start, lookback, df, date_time=True): #Check math here!!
        if date_time:
            start = start.strftime("%Y-%m-%d")
        if ((df.index[df["DATE"]==start][0])-lookback) < 0:
            return None
        df = df[(df.index[df["DATE"]==start][0])-lookback:df.index[df["DATE"]==start][0]+1]
        df = df.reset_index()
        mean_close = df[0:lookback]['CLOSE'].sum() / lookback
        ratio = mean_close / df.iloc[lookback]['CLOSE']
        return ratio

In [11]:
def target(start, look_forward, df, target, date_time=True):
        if date_time:
            start = start.strftime("%Y-%m-%d")
        if ((df.index[df["DATE"]==start][0])+look_forward) > len(df): #this works
            return None
        df = df[(df.index[df["DATE"]==start][0]):df.index[df["DATE"]==start][0]+look_forward] #+1? 9 or 10 days?
        df = df.reset_index()
        percent_increase = (df.iloc[look_forward-1]['CLOSE'] - df.iloc[0]['CLOSE']) / df.iloc[0]['CLOSE'] * 100
        if percent_increase >= target:
            return True
        else:
            return False

In [12]:
def calcutron(stock_data, stock_symbols, l = True):
    df_list = []
    if not l:
        stock_symbols = [stock_symbols]
    for stock in stock_symbols:
        df = stock_data.loc[stock_data["TICKER"]==stock]
        for num in [5,10,30,60]:
            df['RSI ' + str(num)] = df.apply(lambda row: RSI_calc(row['DATE'], num, df, True), axis=1)
            df["AVG DAILY RETURN " + str(num)] = df.apply(lambda row: daily_return(row['DATE'], num, df, True), axis=1)
            df["AVG WEEKLY RETURN " + str(num)] = df.apply(lambda row: weekly_return(row['DATE'], num, df, True), axis=1)
            df["AVG MONTHLY RETURN " + str(num)] = df.apply(lambda row: monthly_return(row['DATE'], num, df, True), axis=1)
            df["RATIO " + str(num)] = df.apply(lambda row: ratio_avg_to_close(row['DATE'], num, df, True), axis=1)
        df["MACD 10_30"] = df.apply(lambda row: MACD(row['DATE'], 10, 30, df, True), axis=1)
        df["MACD 5_10"] = df.apply(lambda row: MACD(row['DATE'], 5, 10, df, True), axis=1)
        df["MACD 2_10"] = df.apply(lambda row: MACD(row['DATE'], 2, 10, df, True), axis=1)
        df['MACD 10_30 DIFF'] = df['MACD 10_30'].diff()
        df['MACD 5_10 DIFF'] = df['MACD 5_10'].diff()
        df['MACD 2_10 DIFF'] = df['MACD 2_10'].diff()
        df["TARGET"] = df.apply(lambda row: target(row['DATE'], 10, df, 5, True), axis=1)
        df_list.append(df)
    stock_data = pd.concat(df_list)
    stock_data['TARGET'].value_counts(normalize=True) * 100
    return stock_data
    stock_data.to_excel("BIG STONK"+".xlsx")

In [ ]:
def trial

In [13]:
#stock_data = calcutron(stock_data, stock_symbols) #Pick one of these
stock_data = pd.read_excel("BIG STONK.xlsx", engine='openpyxl')
#import copy
#stock_data_backup = copy.deepcopy(stock_data)
stock_data = stock_data.drop(['AVG MONTHLY RETURN 5','AVG MONTHLY RETURN 10'], axis=1)
stock_data = stock_data.dropna()
input_patterns = stock_data.loc[:,'RSI 5':'MACD 2_10 DIFF'][60:len(stock_data)-10]
input_label = stock_data['TARGET'][60:len(stock_data)-10].astype(int) #Why are these floats? It bothers me.
print(stock_data)
print(stock_data['TARGET'].value_counts(normalize=True) * 100)

0.0    75.489171
1.0    24.510829
Name: TARGET, dtype: float64


In [20]:
accuracy_trials = []
recall_trials = []
precision_trials = []
average_precision_trials = []

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(input_patterns, input_label, test_size = .33)
    model = XGBClassifier(eval_metric="logloss", use_label_encoder=False)
    model.fit(X_train, y_train)
    accuracy_trials.append(accuracy_score(y_test, model.predict(X_test)) * 100.0)
    precision_trials.append(precision_score(y_test, model.predict(X_test)) * 100.0)
    recall_trials.append(recall_score(y_test, model.predict(X_test)) * 100.0)
    average_precision_trials.append(average_precision_score(y_test, model.predict(X_test)))
    
mean_accuracy = sum(accuracy_trials) / len(accuracy_trials)
mean_precision = sum(precision_trials) / len(precision_trials)
mean_recall= sum(recall_trials) / len(recall_trials)
mean_avg_precision= sum(average_precision_trials) / len(average_precision_trials)

print("Mean Accuracy: {}%".format(mean_accuracy))
print("Mean Precision: {}%".format(mean_precision))
print("Mean Recall: {}%".format(mean_recall))
print(model)

''' #FIX
print("Mean Avg. Precision: {}%".format(mean_avg_precision))
disp = precision_recall_curve(X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'AP={0:0.2f}'.format(mean_avg_precision))\
disp.plot()
'''

Mean Accuracy: 78.15034065102193%
Mean Precision: 63.006256520310984%
Mean Recall: 26.580428270620942%
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='logloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)


' #FIX\nprint("Mean Avg. Precision: {}%".format(mean_avg_precision))\ndisp = precision_recall_curve(X_test, y_test)\ndisp.ax_.set_title(\'2-class Precision-Recall curve: \'\n                   \'AP={0:0.2f}\'.format(mean_avg_precision))disp.plot()\n'

In [21]:

#Probabilities
X_train, X_test, y_train, y_test = train_test_split(input_patterns, input_label, test_size = .33)
model = XGBClassifier(objective="binary:logistic", eval_metric="logloss")
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)
print(y_pred)
'''
#Predictions
X_train, X_test, y_train, y_test = train_test_split(input_patterns, input_label, test_size = .33)
model = XGBClassifier(objective="binary:logistic", eval_metric="logloss")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(y_pred)
'''

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[[0.9873838  0.01261623]
 [0.90679467 0.09320533]
 [0.94497657 0.05502341]
 ...
 [0.94722545 0.05277455]
 [0.732772   0.267228  ]
 [0.9528504  0.04714959]]


'\n#Predictions\nX_train, X_test, y_train, y_test = train_test_split(input_patterns, input_label, test_size = .33)\nmodel = XGBClassifier(objective="binary:logistic", eval_metric="logloss")\nmodel.fit(X_train, y_train)\ny_pred = model.predict(X_test)\nprint(y_pred)\n'

In [ ]:
stock_symbols_testing = ["PEP"]

for stock in stock_symbols_testing:
    stock_data_testing = pd.DataFrame(columns=["Ticker","Date","Open","High","Low","Close"])
    y_finance_data = yf.Ticker(stock).history(start=(datetime.date.today()-datetime.timedelta(days=1825)).strftime("%Y-%m-%d")).reset_index()
    y_finance_data.insert(0,"Ticker", stock)
    y_finance_data = y_finance_data[['Ticker', 'Date','Open','High','Low','Close']]
    
    stock_data_testing = pd.concat([stock_data_testing, y_finance_data])
    stock_data_testing = stock_data_testing.rename(columns={"Ticker":"TICKER", "Date":"DATE", "Open":"OPEN", "High":"HIGH", "Low":"LOW", "Close":"CLOSE"})
    stock_data_testing = calcutron(stock_data_testing, stock, l=False)
    stock_data_testing = stock_data_testing.drop(['AVG MONTHLY RETURN 5','AVG MONTHLY RETURN 10'], axis=1)
    stock_data_testing = stock_data_testing.dropna()
    print(stock_data_testing)
    
    input_patterns_testing = stock_data_testing.loc[:,'RSI 5':'MACD 2_10 DIFF'][60:len(stock_data)-10]
    input_label_testing = stock_data_testing['TARGET'][60:len(stock_data)-10].astype(int)
    
    #print(stock_data_testing)
    #print(input_patterns_testing)
    #print(input_label_testing)
    
    accuracy_trials_testing = []
    recall_trials_testing = []
    precision_trials_testing = []
    #average_precision_trials_testing = []
    
    y_prob = model.predict_proba(input_patterns_testing)
    y_prob_round = []
    threshold = .85
    for row in y_prob:
        if row[1] > threshold:
            y_prob_round.append(1)
        else:
            y_prob_round.append(0)
            
    y_pred = model.predict(input_patterns_testing)
    
    accuracy_trials_testing.append(accuracy_score(input_label_testing, y_pred) * 100.0)
    precision_trials_testing.append(precision_score(input_label_testing, y_pred) * 100.0)
    recall_trials_testing.append(recall_score(input_label_testing, y_pred) * 100.0)
    #average_precision_trials_testing.append(average_precision_score(y_test, model.predict(input_patterns_testing)))

    mean_accuracy_testing = sum(accuracy_trials_testing) / len(accuracy_trials_testing)
    mean_precision_testing = sum(precision_trials_testing) / len(precision_trials_testing)
    mean_recall_testing = sum(recall_trials_testing) / len(recall_trials_testing)
    #mean_avg_precision_testing = sum(average_precision_trials_testing) / len(average_precision_trials_testing)

    print("Mean Accuracy {}: {}%".format(stock, mean_accuracy_testing))
    print("Mean Precision {}: {}%".format(stock, mean_precision_testing))
    print("Mean Recall {}: {}%".format(stock, mean_recall_testing))
    
    accuracy_trials_testing.append(accuracy_score(input_label_testing, y_prob_round) * 100.0)
    precision_trials_testing.append(precision_score(input_label_testing, y_prob_round) * 100.0)
    recall_trials_testing.append(recall_score(input_label_testing, y_prob_round) * 100.0)
    #average_precision_trials_testing.append(average_precision_score(y_test, model.predict(input_patterns_testing)))

    mean_accuracy_testing = sum(accuracy_trials_testing) / len(accuracy_trials_testing)
    mean_precision_testing = sum(precision_trials_testing) / len(precision_trials_testing)
    mean_recall_testing = sum(recall_trials_testing) / len(recall_trials_testing)
    #mean_avg_precision_testing = sum(average_precision_trials_testing) / len(average_precision_trials_testing)

    print("Rounding Threshold: {}".format(threshold))
    print("Mean Accuracy {}: {}%".format(stock, mean_accuracy_testing))
    print("Mean Precision {}: {}%".format(stock, mean_precision_testing))
    print("Mean Recall {}: {}%".format(stock, mean_recall_testing))
    
    ''' #FIX
    print("Mean Avg. Precision: {}%".format(mean_avg_precision))
    disp = precision_recall_curve(X_test, y_test)
    disp.ax_.set_title('2-class Precision-Recall curve: '
                       'AP={0:0.2f}'.format(mean_avg_precision))\
    disp.plot()
    '''